# CEWA572 Practice Midterm

The general form of the advection-diffusion equation is given as:

$$     \frac{\partial c}{\partial t} + u \frac{\partial c}{\partial x} = \kappa \frac{\partial^{2} c}{\partial x^{2}}
$$

where $c$ is the tracer concentration, $u$ is the advective velocity, and $\kappa$ is the diffusion coefficient, or diffusivity.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erfc

### 1.a. (5 pts)

Derive a set of Crank-Nicolson finite difference equations to solve the advection-diffusion equation. Note that the Crank-Nicolson scheme is an average of the forward-time centered-space and backward-time centered-space schemes such that an equation like:

$$     \frac{\partial c}{\partial t} = \kappa \frac{\partial^{2} c}{\partial x^{2}} $$

can be discretized using the "theta method" as:

$$     \frac{c^{k+1}_{i}-c^{k}_{i}}{\Delta t} = \kappa \left [ \theta \left \lbrace \frac{c^{k+1}_{i+1}+c^{k+1}_{i-1}-2c^{k+1}_{i}}{(\Delta x)^{2}} \right \rbrace + (1-\theta) \left \lbrace \frac{c^{k}_{i+1}+c^{k}_{i-1}-2c^{k}_{i}}{(\Delta x)^{2}} \right \rbrace \right ] $$

with $theta$ = 0.5.

**This equation above is just an example, please derive the Crank-Nicolson scheme for the advection-diffusion equation.**

The problem setup for the following parts of the question is as follows.

**Consider a porous rod of length $L$ = 50 m, subject to the initial and boundary conditions:**

$$         c =c_{0} \quad t=0 \quad \text{ for all } \quad 0 \leq x \leq L \\
        c = c_{1} \quad t > 0 \quad \text{ for all } \quad x=0 \\
        c =c_{0} \quad t > 0 \quad \text{ for all } \quad x=L  $$
        
 **with $u$=5.0 m/day, $\kappa$=0.3333 m$^{2}$/day, $c_{0}$=0 mg/L and $c_{1}$=100 mg/L.**

### 1.b. (5 pts)

Use the analytical solution coded below to plot the distribution of the concentration $c$ versus $x$ for $t$ = 1 and 5 days. Note you will need to assign the variables.

In [1]:
C_ana = c0/2 * (erfc((x-u*T)/(2*np.sqrt(kappa*T))) + np.exp(u*x/kappa) * erfc((x+u*T)/(2*np.sqrt(kappa*T))))

NameError: name 'c0' is not defined

### 1.c. (5 pts)

Using the Crank-Nicolson scheme derived in part (a), write the code below to obtain a numerical solution of tracer concentration $c$ variability versus $x$ at $t$ = 1 and 5 days. **Use a $\Delta x$ = 1.0 m and $\Delta t$ = 0.05 day. Plot your results and compare your solution to the analytical solution.**

***

One possible finite difference approximation of the advective term

$$ u\frac{\partial c}{\partial x}$$

is based on using the values of concentration at a spatial node (grid point) and the node directly upgradient, or:

$$ \frac{\partial c}{\partial x} = \frac{c_{i} - c_{i-1}}{\Delta x}$$

### 1.d. (20 points)

Derive the finite difference approximation for the advection-diffusion equation using this upgradient form of the advective term. Use the Crank-Nicolson scheme for the diffusion stepping scheme.

### 1.e. (20 points)

Modify your original Crank-Nicolson program to use the new approximation to solve the problem from part (c). Plot the concentration profiles at days 1 and 5 using the new program and the original program. Note that the new curve exhibits more diffusion than the curves you produced originally and that the oscillations go away.

### 1.f (45 points) 

Why does this new numerical approximation damp oscillations? To determine why and under what conditions this occurs, define the following:

- $\kappa$ is the value used for the diffusion coefficient when using the central-in-space approximation of the advective term
- $\kappa^{*}$ is the value used for the diffusion coefficient when using the up-gradient approximation of the advective term

The relationship between the two is $\kappa = \kappa^{*} - Z$, where $Z$ is another type of diffusion coefficient. 

Come up with a general formula that can be used to determine the value of $Z$. This formula should be a function of known quantities. **Hint:** Check the course notes on the concepts of numerical diffusion (Chapter 6). 

Now calculate $Z$ using the new general formula, and adjust the value of $\kappa$ in your new program accordingly. Show that the results match the original solutions as shown in part (c) and that the oscillations reappear.